- The ipyparallel controller creates two files which contain information how to connect to the controller (address and authentication secrets).
- To to connect our engines and clients to the cluster, we need to copy those files to our local machine.
- We use `rsync`.

The address changes, so replace `1.2.3.4` by the address shown on the projector.

In [ ]:
!rsync -v 'student@1.2.3.4:.ipython/profile_default/security/ipcontroller-*.json' ~/.ipython/profile_default/security/

Those files are just JSON. We can print them with syntax highlighting with `pygmentize`:

In [ ]:
!pygmentize /home/zbyszek/.ipython/profile_default/security/ipcontroller-engine.json 

In [ ]:
!pygmentize /home/zbyszek/.ipython/profile_default/security/ipcontroller-client.json 

This starts `htop` in a terminal window to monitor running processes.

If you prefer, start `htop` manually in some terminal window.

In [ ]:
!bash -c 'urxvt -e htop & sleep 3'

Now we start our ipcluster engines on the local machine.
They will connect to the controller configured by the JSON `ipcontroller-engine.json`. 

In [ ]:
!bash -c 'urxvt -e ipcluster engines & sleep 3'

In [ ]:
import ipyparallel as ipp
rc = ipp.Client()

In [ ]:
rc.ids

In [ ]:
def hostname():
    import socket
    return socket.gethostname()

In [ ]:
rc[:].apply_sync(hostname)

In [ ]:
def f(x):
    return x**4 - 3*x

In [ ]:
def integrate(a, b, n):
    dx = (b - a) / n
    s = 0.0
    x = a + 0.5 * dx
    for i in range(n):
        y = f(x)
        s += y
        x += dx
    return s * dx

In [ ]:
rc[:].push(dict(f=f)).wait()

In [ ]:
stepcounts = list(range(10_000, 20_000, 10))
a = [-2 for _ in stepcounts]
b = [+2 for _ in stepcounts]

In [ ]:
%timeit -r 1 -n 1    list(map(integrate, a, b, stepcounts))

In [ ]:
%timeit -r 1 -n 1    rc[:].map_sync(integrate, a, b, stepcounts)